In [46]:
import re
import numpy as np

def one_hot(index0,size0):
    out0=np.zeros(size0,dtype=np.float32())
    out0[index0]=1.
    return out0

def mean_squared_error(y_true, y_pred):
    return np.mean((y_pred - y_true)**2)

# 1. Input sentences
sentences = [
    "the cat sat on the mat",
    "a small dog ran quickly",
    "the big cat jumped over the dog"
]

vocab_count_dict={}
bigram_list=[]
for sent0 in sentences:
    sent_tokens0=re.findall("\w+",sent0.lower())
    padded_tokens0=["<s>"]+sent_tokens0+["</s>"]
    for tk0 in padded_tokens0:
        vocab_count_dict[tk0]=vocab_count_dict.get(tk0,0)+1
    cur_bigrams0=[(padded_tokens0[i],padded_tokens0[i+1]) for i in range(len(padded_tokens0)-1)]
    bigram_list.extend(cur_bigrams0)
    # print(padded_tokens0)
    # print(cur_bigrams0)
# for a,b in vocab_count_dict.items():
#     print(a,b)

# 2. Preprocess
# corpus = " ".join(sentences).lower().split()
vocab = sorted(list(set(vocab_count_dict.keys() )))
word_to_idx = {w: i for i, w in enumerate(vocab)}
idx_to_word = {i: w for w, i in word_to_idx.items()}
V = len(vocab)

#print(vocab)

# # 3. Predefined features
#features = ["content_word", "function_word"]
features = ["noun", "verb","det","prep","other"]
K = len(features)

# 4. Initialize parameters
np.random.seed(42)
F_raw = np.random.rand(V, K) * 0.01  # raw feature weights before sigmoid - try np.random.randn at some point
T = np.random.rand(K, V) * 0.01      # transition matrix

# print(T.shape)
# print(F_raw)
lr = 0.05  # learning rate

cur_word0="<s>"
target_word0="the"
#cur_tag0="other"
cur_tag0="noun"

tag_i=features.index(cur_tag0)
cur_i=word_to_idx[cur_word0]
target_i=word_to_idx[target_word0]

F_raw[cur_i][tag_i]+=lr
T[tag_i][target_i]+=lr

n_epochs=10

for epoch0 in range(n_epochs):
    print("epoch0",epoch0)
    for big0 in bigram_list[:1]:
        print(big0)
        first0,second0=big0
        first_i0,second_i0=word_to_idx[first0],word_to_idx[second0]
        #print(first_i0,second_i0)
        row0=first_wts=F_raw[first_i0]
        
        ft_wts=[(ft0,round(float(ft_wt0) ,4)) for ft0,ft_wt0 in zip(features,row0)]
        print(ft_wts)
        
        #print(first_wts.shape)
        actual0=one_hot(second_i0,V)
        pred0=first_wts @ T
        loss0=mean_squared_error(pred0,actual0)
        print(pred0)
        print(actual0)
        print(loss0)
    F_raw[cur_i][tag_i]+=lr
    T[tag_i][target_i]+=lr



#final checking
for i0,row0 in enumerate(F_raw):
    word0=idx_to_word[i0]
    ft_wts=[(ft0,round(float(ft_wt0) ,4)) for ft0,ft_wt0 in zip(features,row0)]
    #print(i0,word0,ft_wts)

# # 5. Activation functions
# def sigmoid(x):
#     return 1 / (1 + np.exp(-x))

# def sigmoid_deriv(sig):
#     return sig * (1 - sig)

# def softmax(x):
#     e_x = np.exp(x - np.max(x))
#     return e_x / np.sum(e_x)

# # 6. Training loop
# epochs = 50
# for epoch in range(epochs):
#     total_loss = 0
#     for i in range(len(corpus) - 1):
#         w_curr = word_to_idx[corpus[i]]
#         w_next = word_to_idx[corpus[i + 1]]

#         # Forward pass
#         F_sigmoid = sigmoid(F_raw[w_curr])       # apply sigmoid: shape (K,)
#         logits = F_sigmoid @ T                   # shape (V,)
#         probs = softmax(logits)                  # shape (V,)

#         # Loss (cross-entropy)
#         loss = -np.log(probs[w_next] + 1e-9)
#         total_loss += loss

#         # Backward pass
#         dlogits = probs.copy()
#         dlogits[w_next] -= 1  # dL/dlogits

#         dT = np.outer(F_sigmoid, dlogits)                # dL/dT: (K, V)
#         dF_sigmoid = T @ dlogits                         # dL/d(sigmoid(F_raw)): shape (K,)
#         dF_raw = dF_sigmoid * sigmoid_deriv(F_sigmoid)   # dL/dF_raw using sigmoid chain rule

#         # Parameter updates
#         T -= lr * dT
#         F_raw[w_curr] -= lr * dF_raw

#     if epoch % 10 == 0:
#         print(f"Epoch {epoch}, Loss: {total_loss:.4f}")


epoch0 0
('<s>', 'the')
[('noun', 0.0516), ('verb', 0.0006), ('det', 0.0087), ('prep', 0.006), ('other', 0.0071)]
[0.00046958 0.00052796 0.00017693 0.00024027 0.0001534  0.0005668
 0.00040537 0.00034057 0.00012464 0.0002667  0.00027103 0.00050601
 0.00038951 0.00056165 0.00294987]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
0.06627407070527176
epoch0 1
('<s>', 'the')
[('noun', 0.1016), ('verb', 0.0006), ('det', 0.0087), ('prep', 0.006), ('other', 0.0071)]
[0.00083409 0.00091359 0.00021396 0.0004195  0.00021133 0.00099836
 0.00071702 0.00050602 0.00015642 0.00042219 0.00043363 0.00087081
 0.00070829 0.00100526 0.01076397]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
0.06523960938231262
epoch0 2
('<s>', 'the')
[('noun', 0.1516), ('verb', 0.0006), ('det', 0.0087), ('prep', 0.006), ('other', 0.0071)]
[0.00119859 0.00129923 0.00025098 0.00059873 0.00026927 0.00142991
 0.00102867 0.00067147 0.0001882  0.00057769 0.00059622 0.00123561
 0.00102707 0.00144886 0.02357808]
[0. 0. 0. 0. 0. 0.

In [34]:
0.06664857887212164


np.float32(0.0)